In [32]:
import os
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI

from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [33]:
# Load environment variables
load_dotenv()

# Initialize LLM and memory
LANGCHAIN_TRACING_V2 = "true"
LANGSMITH_ENDPOINT = "https://api.smith.langchain.com"
LANGSMITH_PROJECT = "Information_Extractor"
LANGCHAIN_API_KEY = os.getenv("LANGSMITH_API_KEY")
OPENAI_API_KEY = os.getenv("GROQ_API")


In [29]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=1000)
wiki_tool = WikipediaQueryRun(api_wrapper=api_wrapper)

In [30]:
wiki_tool.run({"query": "Agentic AI"})

'Page: Agentic AI\nSummary: Agentic AI is a class of artificial intelligence that focuses on autonomous systems that can make decisions and perform tasks without human intervention. The independent systems automatically respond to conditions, to produce process results. The field is closely linked to agentic automation, also known as agent-based process management systems, when applied to process automation. Applications include software development, customer support, cybersecurity and business intelligence.'

In [31]:
# Initialize the Groq LLM through LangChain
llm = ChatOpenAI(
    model="llama3-70b-8192",
    base_url="https://api.groq.com/openai/v1",
    openai_api_key=OPENAI_API_KEY,
    temperature=0.0,
)

tools = [wiki_tool]
# llm_with_tools = llm.bind_tools(tools) # this does work directly with free models of Groq till now.

question = "Who is Elon Musk? Use Wikipedia if needed."

# Step 1: Ask LLM to decide tool use
decision_prompt = f"say 'CALL_WIKI' if you want to use a Wikipedia tool. Question: {question}"
decision = llm.invoke(decision_prompt)
print("LLM decision:", decision.content)

# Step 2: Tool call if requested
if "CALL_WIKI" in decision.content:
    print("Wiki tool called...")
    tool_result = wiki_tool.run(question)
    print("Tool Result:", tool_result)
    followup_prompt = f"The Wikipedia tool returned this:\n{tool_result}\nNow give a final answer to the user's question."
else:
    followup_prompt = decision.content  # LLM already gave the answer

# Step 3: Final LLM response
final_response = llm.invoke(followup_prompt)
print("Final answer:", final_response.content)


LLM decision: CALL_WIKI!

According to Wikipedia, Elon Musk is a South African-born entrepreneur, inventor, and business magnate. He is the CEO and CTO of SpaceX, CEO and product architect of Tesla Inc., and co-founder of Neuralink and The Boring Company.

Musk was born in Pretoria, South Africa, and moved to Canada in 1992 to attend college. He transferred to the University of Pennsylvania two years later, where he graduated with a degree in economics and physics. After college, he moved to California to pursue a career in technology and entrepreneurship.

Musk's first company, Zip2, which provided online content publishing software for news organizations. In 1999, he co-founded X.com, which later became PayPal, an online payment system that was acquired by eBay for $1.5 billion in 2002.

In 2002, Musk founded SpaceX, with the goal of reducing space transportation costs and enabling the colonization of Mars. In 2004, he co-founded Tesla Inc., which has become one of the most successfu

In [34]:
OPENAI_API_KEY = os.getenv("OPEN_API")

In [39]:
from langchain_openai import ChatOpenAI

In [ ]:
decision_prompt = f"""
You are an AI agent. If you don't know the answer or need external knowledge, call a tool.
Question: {question}
"""

llm = ChatOpenAI(
    model="gpt-4o-mini",
    openai_api_key=OPENAI_API_KEY,
    temperature=0,
)

tools = [wiki_tool]
llm_with_tools = llm.bind_tools(tools)

result = llm_with_tools.invoke([
    {"role": "user", "content": decision_prompt}
])
result.content

''

In [52]:
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import HumanMessage

agent_executor = create_react_agent(llm, tools)

response = agent_executor.invoke({"messages": [HumanMessage(decision_prompt)]})
response


{'messages': [HumanMessage(content="\nYou are an AI agent. If you don't know the answer or need external knowledge, call a tool.\nQuestion: Who is Elon Musk? Use Wikipedia if needed.\n", additional_kwargs={}, response_metadata={}, id='340e946e-de91-4220-b946-665d51cc5c69'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_Xi5EGRWMwFRJZ4OWUcfI8SYM', 'function': {'arguments': '{"query":"Elon Musk"}', 'name': 'wikipedia'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 115, 'total_tokens': 130, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_62a23a81ef', 'id': 'chatcmpl-BiJFuSHNYj1WktvdrwOottr4wVx13', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, i